# Dota 2 Autoencoder and clustering analyzer

# Autoencoder

### Train/Load Autoencoder

In [ ]:
from dota import Dota2
from model import Dota2Autoencoder
from permutations import run_permutations
dota = Dota2([56], ['professional'], (30, 120))

print("Training models...")
train_df, val_df, test_df = dota.prepare_data_splits(dota.dataset)
best_permutation, loss_path, model_path, plot_path, report_path = run_permutations(
    dota, 100, train_df, val_df, test_df, "dota_autoencoder", "dota_autoencoder_loss", "dota_autoencoder_permutations")
print ("Best permutation:", best_permutation)
print("Loading best model...")
model = Dota2Autoencoder.load_model(model_path)

Inicializando Dota2 Autoencoder...
Carregando dataset...
Tier: ['professional'], Duração: 30-120 minutos
Patches:
Patch 7.36 (56): 10844 partidas
Training models...
Available permutations: 288
Permutations hash: 9990013b7bb6a5c100a6907ae6e6da00ddcc3de972495158b39f3df7440efc6e
Permutation results already exist: ./tmp/9990013b7bb6a5c100a6907ae6e6da00ddcc3de972495158b39f3df7440efc6e/dota_autoencoder_permutations.csv
Best permutation: 0_001_0_3_256__128__64__32_16_4_8 with loss 0.0001
Test Accuracy: 1.0, Avg MSE: 6e-06, Stopped at: 99
Best permutation: {'permutation': '0_001_0_3_256__128__64__32_16_4_8', 'loss': 0.0001, 'accuracy': 1.0, 'avg_mse': 6e-06, 'stopped': 99, 'model_path': './tmp/9990013b7bb6a5c100a6907ae6e6da00ddcc3de972495158b39f3df7440efc6e/dota_autoencoder_0_001_0_3_256__128__64__32_16_4_8.h5', 'loss_path': './tmp/9990013b7bb6a5c100a6907ae6e6da00ddcc3de972495158b39f3df7440efc6e/dota_autoencoder_loss_0_001_0_3_256__128__64__32_16_4_8.csv'}
Loading best model...
Modelo carregad